In [1]:
import tensorflow as tf
import os
import numpy as np

# Convert pcap file to bytes
def pcap_to_bytes(pcap_file):
    with open(pcap_file, 'rb') as f:
        return f.read()

# Get list of pcap files in directory
pcap_dir_password_attack = './password_attack/'
pcap_files_password_attack = [os.path.join(pcap_dir_password_attack, f) for f in os.listdir(pcap_dir_password_attack) if f.endswith('.pcap')]
pcap_files_password_attack = pcap_files_password_attack[200:300]
label_pa = np.full(len(pcap_files_password_attack), 0)

pcap_dir_port_scan = './port_scan/'
pcap_files_port_scan = [os.path.join(pcap_dir_port_scan, f) for f in os.listdir(pcap_dir_port_scan) if f.endswith('.pcap')]
pcap_files_port_scan = pcap_files_port_scan[200:300]
pcap_files = pcap_files_password_attack + pcap_files_port_scan
label_ps = np.full(len(pcap_files_port_scan), 1)
# Convert pcap files to bytes
pcap_bytes = [pcap_to_bytes(f) for f in pcap_files]

# Define labels
labels = np.concatenate((label_pa, label_ps), axis=0)

print(labels.shape)
print(len(pcap_bytes))
print(len(pcap_bytes[0]))
print(len(pcap_bytes[1]))

(200,)
200
1581522
1574172


In [2]:
# Convert pcap bytes to numpy array
max_length = 1643410
pcap_bytes_array = np.zeros((len(pcap_bytes), max_length))
for i, b in enumerate(pcap_bytes):
    padded_b = b.ljust(max_length, b'\x00')
    if len(padded_b) > max_length:
        print(i)
        padded_b = padded_b[:max_length]
    pcap_bytes_array[i, :] = np.frombuffer(padded_b, dtype=np.uint8)
print(pcap_bytes_array[0])

mean = 68.6014175415039
std = 79.07476806640625
pcap_bytes_array = (pcap_bytes_array - mean) / std

4
17
[212. 195. 178. ...   0.   0.   0.]


In [3]:
def reshape_(X):
    return X.reshape(X.shape[0], X.shape[1], 1)

In [4]:
from tensorflow.keras.models import load_model
model = load_model('./model.h5')

# pcap_bytes_array = reshape_(pcap_bytes_array)
yhat = model.predict(pcap_bytes_array,batch_size=32)

7/7 [==============================] - 2s 58ms/step


In [5]:
yhat = np.argmax(yhat, axis=-1)
print(yhat)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [6]:
print(yhat.shape)
print(labels.shape)

(200,)
(200,)


In [7]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(labels, yhat)
print(accuracy)

0.745
